## A1-OpenJij (cxxjij編)

今までのチュートリアルではOpenJijの様々な機能を紹介してきましたが、本チュートリアルではOpenJijの更に深いレイヤーであるcxxjijの紹介をします。

基本的にOpenJijのチュートリアルを一通り読んだ人や量子アニーリング等の数式に多少馴染みがある人向けの内容で、

- 最適化問題だけでなくサンプリングなどより一般的な用途にOpenJijを用いたい
- アニーリングスケジュールの設定や使用するアルゴリズム等を直接触りたい

といった目的に使用できます。

In [42]:
#!pip install openjij --user

In [43]:
!pip show openjij

Name: openjij
Version: 0.0.8
Summary: Framework for the Ising model and QUBO
Home-page: https://openjij.github.io/OpenJij/
Author: Jij Inc.
Author-email: openjij@j-ij.com
License: Apache License 2.0
Location: /home/jiko/.local/lib/python3.7/site-packages
Requires: numpy, debtcollector, requests
Required-by: 


## cxxjijについて

OpenJijはインターフェースはpythonで記述されていますが、内部はC++で実装されています。これらのC++インターフェースを直接呼び出しているインターフェースがcxxjijです。

今までチュートリアルで取り扱ってきたインターフェースはcxxjijを内部で呼び出す形で実装されています。

## Ising model 再訪

以前のチュートリアル (第一章)で紹介したIsingモデル

$$H(\{\sigma_i\}) = \sum_{i > j} J_{ij}\sigma_i \sigma_j + \sum_{i=1}^N h_i \sigma_i$$
$$\sigma_i \in \{-1, 1\}, i=1,\cdots N$$

の基底状態をcxxjijを用いて計算してみましょう。

第一章と同じく、変数の数が$N=5$で

$$h_i = -1~\text{for} ~\forall i, ~ J_{ij} = -1~\text{for} ~\forall i, j$$

の場合を考えます。この場合の基底状態は$\{\sigma_i\} = \{1, 1, 1, 1, 1\}$となります。

cxxjijによる実装では以下のような形式になります。

> cxxjijにはQUBOとの変換は実装されていません。QUBOとの変換を行うには今までのチュートリアルを参照してください。

In [44]:
#cxxjijをインポートするには以下のように記述します。
import cxxjij as cj

#問題を表す縦磁場と相互作用を作ります。
#cxxjijでは専用のグラフを表すクラスを利用します。
import cxxjij.graph as G
N = 5

#密結合グラフを定義します。JijがスパースなモデルではSparseクラスを利用したほうが良いです。
J = G.Dense(N)

#Jij
for i in range(N):
    for j in range(i+1, N):
        J[i,j] = -1.0/N

#hi
for i in range(N):
    J[i] = -1
    #J[i,i] = -1でも同じになります。
    
#問題を解くsystemを定義します。今回は古典イジングスピンモデルを作成します。
import cxxjij.system as S
system = S.make_classical_ising_Eigen(J.gen_spin(), J)

#スケジュールリストを作成します。今回の例では、逆温度$\beta=0.1$から\beta=100まで、温度変化を100回に分けてアニーリングし、各温度で200モンテカルロステップ動かします。
import cxxjij.utility as U
schedule_list = U.make_classical_schedule_list(0.1, 100, 100, 200)

#アニーリングを行います。今回はSingleSpinFlipアルゴリズムを用いてsystemを更新します。
import cxxjij.algorithm as A
A.Algorithm_SingleSpinFlip_run(system, schedule_list)

#結果を取得します。
import cxxjij.result as R
result_spin = R.get_solution(system)

print(result_spin)

[1, 1, 1, 1, 1]


## cxxjijの解説

ここからは上で書いたコードに使用されている機能の詳細な解説をします。cxxjijでは主に`graph, system, algorithm`のモジュールから成り立っており、それぞれのモジュールを組み合わせることで様々な種類、アルゴリズムを用いてイジングモデルを計算することが可能になっており、また新たにアルゴリズムを実装する際に拡張が容易であるという特徴を備えています。

### Graph

イジングハミルトニアンの係数$J_{ij}$を保持するためのモジュールです。基本的に密結合 (全てのJijが0以外の値を持つモデルに適している)を扱う`Dense`と疎結合 (Jijの多くの値が0であるモデルに適している)`Sparse`の2種類が存在します。また、`Sparse`から継承された`Square` (正方格子)、`Chimera` (D-Waveのキメラグラフ)も備えています。内部のC++実装ではそれぞれの構造に合わせて最適なデータ構造で実装されています。

#### `Dense`、`Sparse`

`Dense`と`Sparse`の要素へのアクセス方法は次の通りです。

In [45]:
import cxxjij.graph as G

#密結合 (サイズ10)
dense = G.Dense(10)
#疎結合 (サイズ10)
sparse = G.Sparse(10)

#変数の書き込み
dense[3,8] = 4
dense[3,2] = 4
dense[3,5] = 3
#変数の読み出し
print("J_[3,8] = {}".format(dense[3,8]))
#インデックスはJij = Jjiとなるように設定されています。
print("J_[8,3] = {}".format(dense[8,3]))

#hiにアクセスするには次のようにインデックスを1つだけ指定します。
dense[4] = 10
#hiには、dense[4,4]のようにアクセスすることもできます。
print("J_[4,4] = {}".format(dense[4,4]))

#Sparseも基本的には同じです。

#変数の書き込み
sparse[3,8] = 5
sparse[3,2] = 4
sparse[5,3] = 2
#変数の読み出し
print("J_[3,8] = {}".format(sparse[3,8]))
#インデックスはJij = Jjiとなるように設定されています。
print("J_[8,3] = {}".format(sparse[8,3]))

J_[3,8] = 4.0
J_[8,3] = 4.0
J_[4,4] = 10.0
J_[3,8] = 5.0
J_[8,3] = 5.0


また、次の機能を備えています。

- .adj_nodes(i)
    - ノード番号iの隣接ノードを表示します。
- .gen_spin(seed)
    - ランダムなスピン列を生成します。seedが指定されない場合、ランダムなseedが設定されます
- .calc_energy(spin)
    - 与えられたspinのエネルギーを計算して、出力します。

In [46]:
#ノード3の隣接ノードは2,5,8です。
print(sparse.adj_nodes(3))

[8, 2, 5]


In [47]:
spin = sparse.gen_spin()
print(spin)
print(sparse.calc_energy(spin))

[1, -1, 1, 1, -1, 1, 1, 1, -1, -1]
1.0


> Graphモジュールではパフォーマンスを優先させるためにインデックスの範囲チェック等はデフォルトでは行っておりません。範囲チェックを有効にするにはopenjijをデバッグビルドする必要があります。

#### `Square`、`Chimera`

`Square`、`Chimera`は`Sparse`から継承されているクラスであり、先程の`Dense`や`Sparse`とは多少アクセス方法が異なります。

これら2つのクラスでは、座標と方向を指定することにより$J_{ij}, h_i$にアクセスが可能となります。

`Square`では以下の図のように座標と方向を指定して$J_{ij}, h_i$の設定を行います。図の左上の点が原点$(0,0)$です。

<img src="./square.png" width="500">

In [48]:
#正方格子 (サイズ10x10)
square = G.Square(10,10)

#値の指定 (座標(2,2)から(2,3)に向かう方向)
square[2,2,G.Dir.PLUS_C] = 3
#値の指定 (座標(2,2)から(3,2)に向かう方向)
square[2,2,G.Dir.PLUS_R] = 2
#値の指定 (座標(2,2)から(2,1)に向かう方向)
square[2,2,G.Dir.MINUS_C] = 1
#値の指定 (座標(2,2)から(1,2)に向かう方向)
square[2,2,G.Dir.MINUS_R] = 5
#値の指定 (座標(2,2)のhi)
square[2,2] = -1

#値の取得 (座標(2,3)から(2,2)に向かう方向には3が設定されている)
print(square[2,3,G.Dir.MINUS_C])

3.0


`Chimera`の場合は8つのスピンからなるキメラユニットの位置$(i,j)$と、ユニット内のスピンインデックスの3変数で座標を表します。この座標と方向 (`Square`と同じPLUS_R, MINUS_R, PLUS_C, MINUS_Cに加えてユニット内のスピンへ向かう方向IN_0or4, IN_1or5, IN_2or6, IN_3or7)を用いて$J_{ij}, h_i$を設定します。

<img src="./chimera.png" width="500">

In [49]:
#キメラグラフ (サイズ10x10)
chimera = G.Chimera(10,10)

#値の指定 (座標(2,2,0)から(3,2,0)に向かう方向)
chimera[2,2,0,G.ChimeraDir.PLUS_R] = 3
#値の指定 (座標(2,2,4)から(2,3,4)に向かう方向)
chimera[2,2,4,G.ChimeraDir.PLUS_C] = 5
#値の指定 (座標(2,2,0)から(2,2,5)に向かう方向)
chimera[2,2,0,G.ChimeraDir.IN_1or5] = 4
#値の指定 (座標(2,2,6)から(2,2,3)に向かう方向)
chimera[2,2,6,G.ChimeraDir.IN_3or7] = 1
#値の指定 (座標(2,2,0)のhi)
chimera[2,2,0] = 2

#値の取得 (座標(3,2,0)から(2,2,0)に向かう方向、3が設定されている)
print(chimera[3,2,0,G.ChimeraDir.MINUS_R])
#値の取得 (座標(2,2,3)から(2,2,6)に向かう方向、1が設定されている)
print(chimera[2,2,3,G.ChimeraDir.IN_2or6])
#値の設定 (座標(2,2,0)から(2,3,0)に向かう方向はキメラグラフでは設定できないため、以下のコードはエラーとなる)
#chimera[2,2,0,G.ChimeraDir.PLUS_C] = 3

3.0
1.0


これらのクラスは`Sparse`を継承して作られているため、`Sparse`の機能も使用できます。その際のSparseのインデックスは`to_ind`関数で取得できます。`Sparse`インデックスから`Square`の座標を得るには`to_rc`関数、`Chimera`の座標を得るには`to_rci`関数を用います。

In [50]:
#Sparseへのキャスト
sparse_square = G.Sparse(square)
sparse_chimera = G.Sparse(chimera)

#Square: 座標(2,2)の隣接ノードを表示
nodes = sparse_square.adj_nodes(square.to_ind(2,2))
#Sparseのインデックスで表示される
print(nodes)

[12, 21, 32, 23, 22]


In [51]:
#Squareの座標表示に変換
positions = [square.to_rc(elem) for elem in nodes]
print(positions)

[(1, 2), (2, 1), (3, 2), (2, 3), (2, 2)]


In [52]:
#Chimera: 座標(2,2,0)の隣接ノードを表示
nodes = sparse_chimera.adj_nodes(chimera.to_ind(2,2,0))
#Sparseのインデックスで表示される
print(nodes)

[96, 256, 180, 181, 182, 183, 176]


In [53]:
#Chimeraの座標表示に変換
positions = [chimera.to_rci(elem) for elem in nodes]
print(positions)

[(1, 2, 0), (3, 2, 0), (2, 2, 4), (2, 2, 5), (2, 2, 6), (2, 2, 7), (2, 2, 0)]


In [54]:
#Chimera: 座標(2,2,4)の隣接ノードを表示
nodes = sparse_chimera.adj_nodes(chimera.to_ind(2,2,4))
#Sparseのインデックスで表示される
print(nodes)

[172, 176, 177, 178, 179, 188, 180]


In [55]:
#Chimeraの座標表示に変換
positions = [chimera.to_rci(elem) for elem in nodes]
print(positions)

[(2, 1, 4), (2, 2, 0), (2, 2, 1), (2, 2, 2), (2, 2, 3), (2, 3, 4), (2, 2, 4)]


## System

Systemではモンテカルロ等の計算における、現在のシステムの状態を保持するためのデータ構造が定義されています。具体的には

- 古典イジングモデル (スピン配列)
- 横磁場イジングモデル (トロッター分解も含んだスピン配列)
- GPU実装古典、量子イジングモデル

等が定義されています。モンテカルロ法を始めとする計算手法には極めて様々な手法がある (もしくは今後新しい手法が開発されていく)ため、OpenJijでは各々の計算手法に対応するデータ構造 (System)とアルゴリズム (Updater, Algorithm)、また計算結果の取得(Result)を分離することにより、様々なアルゴリズムを追加することが容易に行えるように設計されています。

Systemを初期化するためには、初期スピン配列とGraphモジュールで作成された$J_{ij}$の2つが必要となります。Systemの種類に応じて初期化に用いることのできるGraphの種類が異なるので注意してください。

### 古典イジングモデル

OpenJijには古典イジングモデルの実装は2種類あり、最もオーソドックスな実装である`ClassicalIsing`と、線形代数ライブラリEigenを内部で用いて高速化を図ったバージョンの2つが存在します。次のように初期化を行います。

In [56]:
import cxxjij.system as S

#古典イジングモデル (Dense, Sparseグラフから)
classical_ising = S.make_classical_ising(dense.gen_spin(), dense)
#古典イジングモデル (スピン、Jijを線形代数ライブラリEigen型で保持)
classical_ising_eigen = S.make_classical_ising_Eigen(dense.gen_spin(), dense)

### 横磁場イジングモデル

$$H = s \left( \sum_{i > j} J_{ij}\sigma_i^z \sigma_j^z + \sum_{i=1}^N h_i \sigma_i^z \right) + \Gamma (1-s) \sum_{i=1}^{N} \sigma_i^x$$

で定義される横磁場イジングモデルを定義するには、初期スピンと$Jij$の他に、$\Gamma$の値とトロッター分割数が必要となります。ここでは分割数を10に設定しています。

In [57]:
gamma = 1.0
trotter_num = 10

#横磁場イジングモデル (Dense, Sparseグラフから)
transverse_ising = S.make_transverse_ising(dense.gen_spin(), dense, gamma, trotter_num)
#横磁場イジングモデル (スピン、Jijを線形代数ライブラリEigen型で保持)
transverse_ising_eigen = S.make_transverse_ising_Eigen(dense.gen_spin(), dense, gamma, trotter_num)

### GPU

GPUを用いて実装されたものも存在します。現時点ではChimeraグラフのデータ構造のみ実装されているので、初期化に必要なGraphは`ChimeraGPU`クラスで作成されたものを用意する必要があります。

> `ChimeraGPU`クラスは`Chimera`クラスとほぼ同じですが、浮動小数点の型が内部で異なるため (c++実装においてデフォルトではcpuではdouble型、gpuではfloat型を利用しています)、別々に実装されています。

In [58]:
#キメラグラフ (サイズ10x10)
chimera = G.ChimeraGPU(10,10)

#値の指定 (座標(2,2,0)から(3,2,0)に向かう方向)
chimera[2,2,0,G.ChimeraDir.PLUS_R] = 3
#値の指定 (座標(2,2,4)から(2,3,4)に向かう方向)
chimera[2,2,4,G.ChimeraDir.PLUS_C] = 5
#値の指定 (座標(2,2,0)から(2,2,5)に向かう方向)
chimera[2,2,0,G.ChimeraDir.IN_1or5] = 4
#値の指定 (座標(2,2,6)から(2,2,3)に向かう方向)
chimera[2,2,6,G.ChimeraDir.IN_3or7] = 1
#値の指定 (座標(2,2,0)のhi)
chimera[2,2,0] = 2

#GPU古典イジングモデル (Chimeraグラフから)
chimera_ising_gpu = S.make_chimera_classical_gpu(chimera.gen_spin(), chimera)
#GPU横磁場イジングモデル (Chimeraグラフから)
chimera_ising_transverse_gpu = S.make_chimera_transverse_gpu(chimera.gen_spin(), chimera, gamma, trotter_num)

## スケジュール設定

アニーリングスケジュールの設定はUtilityモジュールから設定できます。古典イジングモデルを扱うか、量子イジングモデルを扱うかで初期化の仕方が異なります。

### 古典モデル

以下のように逆温度$\beta$のスケジュール設定を行います。

In [59]:
#スケジュールリストを作成します。今回の例では、逆温度$\beta=0.1$から\beta=100まで、温度変化を10回に分けてアニーリングし、各温度で20モンテカルロステップ動かします。
import cxxjij.utility as U
schedule_list = U.make_classical_schedule_list(0.1, 100, 10, 20)

`print`関数でスケジュールの詳細な出力を得ることができます。

In [64]:
print(schedule_list)

[((beta: 0.100000) mcs: 10), ((beta: 0.143845) mcs: 10), ((beta: 0.206914) mcs: 10), ((beta: 0.297635) mcs: 10), ((beta: 0.428133) mcs: 10), ((beta: 0.615848) mcs: 10), ((beta: 0.885867) mcs: 10), ((beta: 1.274275) mcs: 10), ((beta: 1.832981) mcs: 10), ((beta: 2.636651) mcs: 10), ((beta: 3.792690) mcs: 10), ((beta: 5.455595) mcs: 10), ((beta: 7.847600) mcs: 10), ((beta: 11.288379) mcs: 10), ((beta: 16.237767) mcs: 10), ((beta: 23.357215) mcs: 10), ((beta: 33.598183) mcs: 10), ((beta: 48.329302) mcs: 10), ((beta: 69.519280) mcs: 10), ((beta: 100.000000) mcs: 10)]


### 横磁場モデル

以下のように逆温度$\beta$とアニーリングスケジュール$s$の設定を行います。

In [65]:
#逆温度$\beta=10$で、アニーリングスケジュールsを0から1まで100回に分けてアニーリングし、各スケジュールで200モンテカルロステップ動かします。
transeverse_schedule_list = U.make_transverse_field_schedule_list(10, 100, 200)

> スケジュールの設定は現時点では上の手法での初期化でしか対応しておらず、自由にスケジュールを設定する機能はC++のPythonラッパーがまだできていないので、現段階では未実装です。

## Algorithm

作成されたSystemとアニーリングスケジュールをAlgorithmに渡すことで、実際にアニーリングを行います。サンプルコードは以下になります。

In [66]:
import cxxjij.algorithm as A

#SingleSpinFlipアルゴリズムを、設定したアニーリングスケジュールで動かす
A.Algorithm_SingleSpinFlip_run(classical_ising, schedule_list)
#シード値を設定することも可能です。
A.Algorithm_SingleSpinFlip_run(classical_ising, 1234, schedule_list)
#線形代数ライブラリEigenで高速化されたSystemを使うことも可能です。
A.Algorithm_SingleSpinFlip_run(classical_ising_eigen, schedule_list)
#ほぼ同じコードでSwendsen-Wangアルゴリズムを用いることもできます。
A.Algorithm_SwendsenWang_run(classical_ising, schedule_list)
#量子モンテカルロもほぼ同じコードで実行できます。
A.Algorithm_SingleSpinFlip_run(transverse_ising, transeverse_schedule_list)
#GPU版も同じです。GPU版のSystemを実行できます。
A.Algorithm_GPU_run(chimera_ising_gpu, schedule_list)

## Result

アニーリングされて出てきた結果を受け取るモジュールです。`get_solution`関数を用いて結果の取得が可能です。このモジュールにより、古典、量子に関係なく統一された形式で答えを受け取ることができます。

In [68]:
import cxxjij.result as R

#古典
print(R.get_solution(classical_ising))
#量子
print(R.get_solution(transverse_ising))

[1, 1, -1, 1, -1, -1, -1, -1, -1, -1]
[1, -1, 1, -1, -1, 1, -1, -1, 1, -1]
